# Implied Volatility Calculation

Reference: https://www.youtube.com/watch?v=Jpy3iCsijIU

In [3]:
from math import sqrt, exp, log, pi
from scipy.stats import norm

def d(sigma, S, K, r, t):
    d1 = 1 / (sigma * sqrt(t)) * (log(S/K) + (r + sigma**2/2) * t)
    d2 = d1 - sigma * sqrt(t)
    return d1, d2

def call_price(sigma,S,K,r,t,d1,d2):
    C = norm.cdf(d1) * S - norm.cdf(d2) * K * exp(-r * t)
    return C

# Option parameters
# S = 100.0
# K = 105.0
# t = 30.0 / 365.0
# r = 0.01
# C0 = 2.30

# S = 194.11 # stock price
# K = 210.0 # strike price
# t = 38 / 365.0 # time to expiration (years)
# r = 0.01 # risk-free rate (annualized)
# C0 = 1.50

S = 412.95 # stock price
K = 413.0 # strike price
t = 13 / 365.0 # time to expiration (years)
r = 0.01 # risk-free rate (annualized)
C0 = 5.90 # option price


# Tolerances
tol = 1e-3
epsilon = 1

count = 0
max_iter = 1000

vol = 0.50

while epsilon > tol:
    count += 1
    if count >= max_iter:
        print('Breaking on count')
        break;
    
    orig_vol = vol
    
    d1, d2 = d(vol, S, K, r, t)
    function_value = call_price(vol, S, K, r, t, d1, d2) - C0
    
    vega = S * norm.pdf(d1) * sqrt(t) # Greek nu is vega
    vol = -function_value / vega + vol
    
    epsilon = abs( (vol - orig_vol) / orig_vol )

print('Sigma = ',vol) # Sigma is implied volatility
print('Code took ', count, ' iterations')
# d1, d2 = d(0.15, S, K, r, t) # used to test code calculating correctly
# print( call_price(0.15,S,K,r,t,d1,d2)) # used to test code calculating correctly

Sigma =  0.1882331957011529
Code took  2  iterations


# Realized Volatility Calculation

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf # https://pypi.org/project/yfinance/

In [9]:
SPY = yf.download('SPY',period='2y',interval='1d')
SPY['PCT'] = SPY['Close'].pct_change()
SPY_pct = SPY['PCT'].iloc[1:]

vol_pandas = SPY_pct.rolling(20).std().dropna()
print(vol_pandas)

[*********************100%***********************]  1 of 1 completed
Date
2019-10-02    0.006947
2019-10-03    0.006522
2019-10-04    0.007306
2019-10-07    0.007347
2019-10-08    0.008043
                ...   
2021-08-30    0.005386
2021-08-31    0.005204
2021-09-01    0.005007
2021-09-02    0.004891
2021-09-03    0.004902
Name: PCT, Length: 486, dtype: float64


In [12]:
spy_pct = SPY_pct.to_numpy()
np.std(spy_pct[0:20], ddof = 1)

0.00694650811327982